In [ ]:
# ✅ VERIFY INSTALLATION
# Since you have uploaded the .whl to your Fabric Environment, it should be installed automatically.
# Run this cell to confirm the correct version (v0.1.1) is loaded.

import importlib.metadata

try:
    version = importlib.metadata.version("usf_fabric_monitoring")
    print(f"✅ Library found: usf_fabric_monitoring v{version}")
    
    if version == "0.1.1":
        print("   You are using the correct version.")
    else:
        print(f"⚠️  WARNING: Expected v0.1.1 but found v{version}.")
        print("   Please check your Fabric Environment settings and ensure the new wheel is published.")
        
except importlib.metadata.PackageNotFoundError:
    print("❌ Library NOT found.")
    print("   Please ensure you have attached the 'Fabric Environment' containing the .whl file to this notebook.")
    print("   Alternatively, upload the .whl file to the Lakehouse 'Files' section and pip install it from there.")

# Monitor Hub Analysis Pipeline

## Overview
This notebook executes the **Monitor Hub Analysis Pipeline**, which is designed to provide deep insights into Microsoft Fabric activity. It extracts historical data, calculates key performance metrics, and generates comprehensive reports to help identify:
- Constant failures and reliability issues.
- Excess activity by users, locations, or domains.
- Historical performance trends over the last 90 days.

## How to Use
1. **Install Package**: The first cell installs the `usf_fabric_monitoring` package into the current session.
2. **Configure Credentials**: Ensure your Service Principal credentials (`AZURE_CLIENT_ID`, `AZURE_CLIENT_SECRET`, `AZURE_TENANT_ID`) are available. The notebook attempts to load them from a `.env` file in the Lakehouse or you can configure Azure Key Vault.
3. **Set Parameters**:
    - `DAYS_TO_ANALYZE`: Number of days of history to fetch (default: 90).
    - `OUTPUT_DIR`: Lakehouse path where reports will be saved.
4. **Run Analysis**: Execute the pipeline cell. It will:
    - Fetch data from Fabric APIs.
    - Process and enrich the data.
    - Save CSV reports to the specified `OUTPUT_DIR`.

In [ ]:
from usf_fabric_monitoring.core.pipeline import MonitorHubPipeline
import os

In [ ]:
import inspect
import usf_fabric_monitoring

print(f"📦 Package Location: {os.path.dirname(usf_fabric_monitoring.__file__)}")

# Verify we are running the NEW code (v0.1.1)
try:
    src = inspect.getsource(MonitorHubPipeline._prepare_extraction_directory)
    if "raw_data" in src:
        print("✅ SUCCESS: You are running the updated code (v0.1.1).")
        print("   Output will go to: .../raw_data/ (Persistent Cache)")
    else:
        print("❌ WARNING: You are still running the OLD code (v0.1.0).")
        print("   Output will go to: .../extracted/TIMESTAMP/ (Legacy)")
        print("   👉 ACTION: Restart the kernel and run the install cell above again.")
except Exception as e:
    print(f"⚠️ Could not verify source code: {e}")

In [ ]:
import os
from dotenv import load_dotenv

# --- CREDENTIAL MANAGEMENT ---

# Option 1: Load from .env file in Lakehouse (Easiest migration)
# Upload your .env file to the 'Files' section of your Lakehouse
ENV_PATH = "/lakehouse/default/Files/dot_env_files/.env"
if os.path.exists(ENV_PATH):
    print(f"Loading configuration from {ENV_PATH}")
    load_dotenv(ENV_PATH)
else:
    print(f"Warning: No .env file found at {ENV_PATH}")

# Option 2: Load from Azure Key Vault (Best Practice)
# Uncomment and configure this section to use Azure Key Vault
# try:
#     from notebookutils import mssparkutils
#     KEY_VAULT_NAME = "YourKeyVaultName"
#     os.environ["AZURE_CLIENT_ID"] = mssparkutils.credentials.getSecret(KEY_VAULT_NAME, "Fabric-Client-ID")
#     os.environ["AZURE_CLIENT_SECRET"] = mssparkutils.credentials.getSecret(KEY_VAULT_NAME, "Fabric-Client-Secret")
#     os.environ["AZURE_TENANT_ID"] = mssparkutils.credentials.getSecret(KEY_VAULT_NAME, "Fabric-Tenant-ID")
# except ImportError:
#     pass # Not running in Fabric or notebookutils not available
# except Exception as e:
#     print(f"Key Vault access failed: {e}")

# Verify credentials are present
required_vars = ["AZURE_CLIENT_ID", "AZURE_CLIENT_SECRET", "AZURE_TENANT_ID"]
missing = [v for v in required_vars if not os.getenv(v)]
if missing:
    print(f"❌ Missing required environment variables: {', '.join(missing)}")
else:
    print("✅ Credentials configured successfully")

In [ ]:
# Configuration
DAYS_TO_ANALYZE = 28
OUTPUT_DIR = "/lakehouse/default/Files/monitor_hub_analysis" # Adjust for Fabric Lakehouse path if needed
# Or use local path for testing
# OUTPUT_DIR = "exports/monitor_hub_analysis"

In [ ]:
pipeline = MonitorHubPipeline(OUTPUT_DIR)
results = pipeline.run_complete_analysis(days=DAYS_TO_ANALYZE)
pipeline.print_results_summary(results)